In [51]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re

In [14]:
#ejemplo 'texto / text' -> 'texto'
def reducir_string(texto):
    if isinstance(texto, str):
        return texto.split(' / ')[0]
    else:
        return texto

In [15]:
ex_file = pd.ExcelFile('CNA Alacero 2023-2024.xlsx')

In [53]:
def procesar_hoja(nombre_hoja, archivo_excel, año = True, tipo = True):
    df = archivo_excel.parse(nombre_hoja, skiprows=1, usecols=list(range(1,15))) #lee la página omite la primera fila y selecciona las columnas de la 1 a la 14
    #cambiar nombre de columnas
    df.columns = ['País', 'Producto', 'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'] 
    df = df[(df['Producto'] != 'Total') & (~df['Producto'].astype(str).str.startswith('Var'))] #quitamos filas que no nos interesan (Total y Var)
    

    df['País'] = df['País'].apply(reducir_string) #aplicamos la función a la columna País ex. "Brasil / Brazil" -> "Brasil"
    df['Producto'] = df['Producto'].apply(reducir_string) #aplicamos la función a la columna Producto
    df['País'] = df['País'].ffill() #rellenamos los valores nulos hacia adelante causados por excel

    # Encuentra el índice donde aparece "América Latina"
    indice = df[df['País'] == 'América Latina'].index

    # Si se encontró, corta el DataFrame hasta antes de esa fila
    if not indice.empty:
        df = df.loc[:indice[0] - 1]


    # Transformamos el DataFrame de formato ancho a formato largo usando pd.melt
    df = pd.melt(df, id_vars=['País', 'Producto'], 
                        value_vars=['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'], 
                        var_name='Mes', value_name='Producción')

    if año:
        # Extraer el año con regex
        match = re.search(r'20\d{2}', nombre_hoja)
        año = int(match.group()) if match else None
        df['Año'] = año

    if tipo:
        # Detectar el tipo según el nombre
        if 'Acero' in nombre_hoja:
            tipo = 'Acero'
        elif 'Hierro' in nombre_hoja:
            tipo = 'Hierro'
        elif 'Laminados' in nombre_hoja:
            tipo = 'Laminados'
        else:
            tipo = nombre_hoja



        # Agregar columnas al DataFrame
        df['Material'] = tipo
    

    # Mostramos las primeras filas del DataFrame transformado
    df.head()


    return df

In [54]:
def eda_comercio_acero(df, columna_valor):
    """
    Realiza un EDA interactivo para tablas de importaciones, exportaciones o consumo.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con columnas 'Año', 'Mes', 'País', 'Producto', y una columna de valores.
        columna_valor (str): Nombre de la columna numérica a analizar (ej. 'Importacion', 'Exportacion', 'Consumo').
    """

    # === TRADUCCIÓN DE MESES EN ESPAÑOL A INGLÉS PARA CONVERTIR A FECHA ===
    meses_es_a_en = {
        'Enero': 'January', 'Febrero': 'February', 'Marzo': 'March',
        'Abril': 'April', 'Mayo': 'May', 'Junio': 'June',
        'Julio': 'July', 'Agosto': 'August', 'Septiembre': 'September',
        'Octubre': 'October', 'Noviembre': 'November', 'Diciembre': 'December'
    }

    df['Mes_EN'] = df['Mes'].map(meses_es_a_en)
    df['Fecha'] = pd.to_datetime(df['Año'].astype(str) + "-" + df['Mes_EN'], format="%Y-%B")

    # === 1. SERIES DE TIEMPO: VALOR MENSUAL POR PAÍS ===
    df_tiempo = df.groupby(['Fecha', 'País'])[columna_valor].sum().reset_index()

    fig_tiempo = px.line(df_tiempo,
                         x="Fecha", y=columna_valor,
                         color="País",
                         markers=True,
                         title=f"{columna_valor} mensual por país",
                         labels={columna_valor: f"{columna_valor} (mil toneladas)", "Fecha": "Mes"})

    fig_tiempo.update_layout(template="plotly_white")
    fig_tiempo.show()

    # === 2. PROMEDIO POR PRODUCTO Y PAÍS ===
    df_avg = df.groupby(["País", "Producto"])[columna_valor].mean().reset_index()

    fig_barras = px.bar(df_avg,
                        x="País", y=columna_valor, color="Producto",
                        barmode="group",
                        title=f"Promedio de {columna_valor.lower()} por producto y país",
                        labels={columna_valor: f"Promedio de {columna_valor} (mil toneladas)"})

    fig_barras.update_layout(template="plotly_white")
    fig_barras.show()

    # === 3. BOX PLOT POR PRODUCTO ===
    fig_box = px.box(df,
                     y="Producto", x=columna_valor, color="Producto",
                     orientation='h',
                     title=f"Distribución de {columna_valor.lower()} por tipo de producto",
                     labels={columna_valor: f"{columna_valor} (mil toneladas)"})

    fig_box.update_layout(template="plotly_white")
    fig_box.show()


# PRODUCCIÓN

In [18]:
hojas_produccion = ['Producción Acero Crudo 2024',
 'Producción Acero Crudo 2023',
 'Producción Acero Crudo 2022',
 'Producción Acero Crudo 2021',
 'Producción Acero Crudo 2020',
 'Producción Acero Crudo 2019',
 'Producción Acero Crudo 2018',
 'Producción Acero Crudo 2017',
 'Producción Acero Crudo 2016',
 'Producción Acero Crudo 2015',
 'Producción Acero Crudo 2014',
 'Producción Acero Crudo 2013',
 'Producción Acero Crudo 2012',
 'Producción Acero Crudo 2011',
 'Hierro Primario 2024',
 'Hierro Primario 2023',
 'Hierro Primario 2022',
 'Hierro Primario 2021',
 'Hierro Primario 2020',
 'Hierro Primario 2019',
 'Hierro Primario 2018',
 'Hierro Primario 2017',
 'Hierro Primario 2016',
 'Hierro Primario 2015',
 'Hierro Primario 2014',
 'Hierro Primario 2013',
 'Hierro Primario 2012',
 'Hierro Primario 2011',
 'Producción Laminados 2024',
 'Producción Laminados 2023',
 'Producción Laminados 2022',
 'Producción Laminados 2021',
 'Producción Laminados 2020',
 'Producción Laminados 2019',
 'Producción Laminados 2018',
 'Producción Laminados 2017',
 'Producción Laminados 2016',
 'Producción Laminados 2015',
 'Producción Laminados 2014',
 'Producción Laminados 2013',
 'Producción Laminados 2012',
 'Producción Laminados 2011',]

In [19]:
#borrar variable df_produccion si existe
try:
    del df_produccion
except NameError:
    pass

#bucle para recorrer las hojas de produccion
for hoja in hojas_produccion:
    df_aux = procesar_hoja(hoja, ex_file)
    if 'df_produccion' in locals():
        df_produccion = pd.concat([df_produccion, df_aux], ignore_index=True) #concatenamos los dataframes
    else:
        df_produccion = df_aux #si no existe lo creamos    

In [20]:
df_produccion = df_produccion[['Año', 'Mes', 'País', 'Material', 'Producto', 'Producción']]
df_produccion

,Año,Mes,País,Material,Producto,Producción
0,2024,Enero,Argentina,Acero,H.Eléctrico,240.998
1,2024,Enero,Argentina,Acero,BOF,166.765
2,2024,Enero,Brasil,Acero,H.Eléctrico,656.539
3,2024,Enero,Brasil,Acero,BOF,2066.456
4,2024,Enero,Chile,Acero,H.Eléctrico,35.640
...,...,...,...,...,...,...
17851,2011,Diciembre,Uruguay,Laminados,Terminados,6.247
17852,2011,Diciembre,Venezuela,Laminados,Largos,95.112
17853,2011,Diciembre,Venezuela,Laminados,Planos,20.665
17854,2011,Diciembre,Venezuela,Laminados,Tubos Sin Costura,0.000


In [21]:
df_produccion['Producto'].unique() 

array(['H.Eléctrico', 'BOF', 'Arrabio', 'Hierro Esponja', 'Largos',
       'Planos', 'Tubos Sin Costura', 'Laminados', 'Terminados'],
      dtype=object)

In [22]:
df_produccion.to_csv('limpias/produccion.csv', index=False)  # guardamos el dataframe en un csv

In [58]:
eda_comercio_acero(df_produccion, 'Producción') #llamamos a la función de EDA

# IMPORTACIONES

In [43]:
hojas_importaciones = ['Impo 2024', 'Impo 2023', 'Impo 2022', 'Impo 2021', 'Impo 2020', 'Impo 2019',
                       'Impo 2018', 'Impo 2017', 'Impo 2016', 'Impo 2015', 'Impo 2014']

In [44]:
#borrar variable df_importaciones si existe
try:
    del df_importaciones
except NameError:
    pass

#bucle para recorrer las hojas de produccion
for hoja in hojas_importaciones:
    df_aux = procesar_hoja(hoja, ex_file, año=True, tipo=False)
    if 'df_importaciones' in locals():
        df_importaciones = pd.concat([df_importaciones, df_aux], ignore_index=True) #concatenamos los dataframes
    else:
        df_importaciones = df_aux #si no existe lo creamos  

df_importaciones = df_importaciones[['Año', 'Mes', 'País', 'Producto', 'Producción']]
df_importaciones.rename(columns={'Producción': 'Importacion'}, inplace=True) 
df_importaciones

,Año,Mes,País,Producto,Importacion
0,2024,Enero,Argentina,Largos,9.491000
1,2024,Enero,Argentina,Planos,39.365000
2,2024,Enero,Argentina,Tubos Sin Costura,3.343000
3,2024,Enero,Argentina,Laminados,52.199000
4,2024,Enero,Bolivia,Largos,19.502272
...,...,...,...,...,...
8443,2014,Diciembre,Uruguay,Laminados,8.821937
8444,2014,Diciembre,Venezuela,Largos,6.420720
8445,2014,Diciembre,Venezuela,Planos,43.169088
8446,2014,Diciembre,Venezuela,Tubos Sin Costura,3.008558


In [45]:
df_importaciones['Producto'].unique() 

array(['Largos', 'Planos', 'Tubos Sin Costura', 'Laminados'], dtype=object)

In [46]:
df_importaciones.to_csv('limpias/importaciones_laminados.csv', index=False)  # guardamos el dataframe en un csv

In [55]:
eda_comercio_acero(df_importaciones, 'Importacion')

# EXPORTACIONES

In [31]:
hojas_exportaciones = ['Expo 2024', 'Expo 2023', 'Expo 2022', 'Expo 2021', 'Expo 2020', 'Expo 2019',
                       'Expo 2018', 'Expo 2017', 'Expo 2016', 'Expo 2015', 'Expo 2014']

In [32]:
#borrar variable df_exportaciones si existe
try:
    del df_exportaciones
except NameError:
    pass

#bucle para recorrer las hojas de produccion
for hoja in hojas_exportaciones:
    df_aux = procesar_hoja(hoja, ex_file, año=True, tipo=False)
    if 'df_exportaciones' in locals():
        df_exportaciones = pd.concat([df_exportaciones, df_aux], ignore_index=True) #concatenamos los dataframes
    else:
        df_exportaciones = df_aux #si no existe lo creamos  

df_exportaciones = df_exportaciones[['Año', 'Mes', 'País', 'Producto', 'Producción']]
df_exportaciones.rename(columns={'Producción': 'Exportacion'}, inplace=True) 
df_exportaciones

,Año,Mes,País,Producto,Exportacion
0,2024,Enero,Argentina,Largos,7.79
1,2024,Enero,Argentina,Planos,29.406
2,2024,Enero,Argentina,Tubos Sin Costura,16.536
3,2024,Enero,Argentina,Laminados,53.732
4,2024,Enero,Bolivia,Largos,0.000192
...,...,...,...,...,...
8443,2014,Diciembre,Uruguay,Laminados,0.003223
8444,2014,Diciembre,Venezuela,Largos,0.0
8445,2014,Diciembre,Venezuela,Planos,7.925234
8446,2014,Diciembre,Venezuela,Tubos Sin Costura,0.0


In [33]:
df_exportaciones['Producto'].unique() 

array(['Largos', 'Planos', 'Tubos Sin Costura', 'Laminados'], dtype=object)

In [34]:
df_exportaciones.to_csv('limpias/exportaciones_laminados.csv', index=False)  # guardamos el dataframe en un csv

In [35]:
df_exportaciones['Producto'].unique()

array(['Largos', 'Planos', 'Tubos Sin Costura', 'Laminados'], dtype=object)

In [56]:
eda_comercio_acero(df_exportaciones, 'Exportacion')

# CONSUMO APARENTE

In [36]:
hojas_consumo = ['Consumo Aparente 2024',
 'Consumo Aparente 2023',
 'Consumo Aparente 2022',
 'Consumo Aparente 2021',
 'Consumo Aparente 2020',
 'Consumo Aparente 2019',
 'Consumo Aparente 2018',
 'Consumo Aparente 2017',
 'Consumo Aparente 2016',
 'Consumo Aparente 2015',
 'Consumo Aparente 2014']

In [37]:
#borrar variable df_consumo si existe
try:
    del df_consumo
except NameError:
    pass

#bucle para recorrer las hojas de produccion
for hoja in hojas_consumo:
    df_aux = procesar_hoja(hoja, ex_file, año=True, tipo=False)
    if 'df_consumo' in locals():
        df_consumo = pd.concat([df_consumo, df_aux], ignore_index=True) #concatenamos los dataframes
    else:
        df_consumo = df_aux #si no existe lo creamos  

df_consumo = df_consumo[['Año', 'Mes', 'País', 'Producto', 'Producción']]
df_consumo.rename(columns={'Producción': 'Consumo'}, inplace=True)
df_consumo

,Año,Mes,País,Producto,Consumo
0,2024,Enero,Argentina,Largos,114.844000
1,2024,Enero,Argentina,Planos,158.637000
2,2024,Enero,Argentina,Tubos Sin Costura,29.207000
3,2024,Enero,Argentina,Laminados,302.688000
4,2024,Enero,Bolivia,Largos,19.502080
...,...,...,...,...,...
8971,2014,Diciembre,Uruguay,Laminados,13.569425
8972,2014,Diciembre,Venezuela,Largos,83.723720
8973,2014,Diciembre,Venezuela,Planos,84.201854
8974,2014,Diciembre,Venezuela,Tubos Sin Costura,3.008558


In [38]:
df_consumo['Producto'] = df_consumo['Producto'].replace({
    'Largos *': 'Largos',
    'Planos *': 'Planos',
    'Laminados *': 'Laminados',
    'Planos ***': 'Planos',
    'Tubos Sin Costura **/ Seamless Tubes **': 'Tubos Sin Costura'
})

In [39]:
df_consumo['Producto'].unique()

array(['Largos', 'Planos', 'Tubos Sin Costura', 'Laminados'], dtype=object)

In [40]:
df_consumo.to_csv('limpias/consumo_laminados.csv', index=False)  # guardamos el dataframe en un csv

In [57]:
eda_comercio_acero(df_consumo, 'Consumo')